In [ ]:
from keras.layers import Dense, Activation, Conv2D
from keras.layers import MaxPooling2D, Dropout, Flatten,GlobalMaxPooling2D
from keras.models import Sequential
import random
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import BatchNormalization
# from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
# from keras.layers import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def build_model(layer_dims, input_shape=(224,224,3,),len_classes=2, dropout_rate=0.2,activation='relu'):
    print(1)
    """Function to build a model with specified layer dimensions and activation function."""
    model = Sequential()
    for i, dim in enumerate(layer_dims):
        if i == 0:
            model.add(Conv2D(dim[0], dim[1], input_shape=input_shape, activation=activation))
            model.add(MaxPooling2D(pool_size=(2, 2)))
        else:
            model.add(Conv2D(dim[0], dim[1], activation=activation))
            model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=len_classes, kernel_size=1, strides=1))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(GlobalMaxPooling2D())
    model.add(Activation('softmax'))
    return model

In [ ]:
def sample_architecture(min_layers=1, max_layers=5, min_filters=32, max_filters=512, min_kernel=3, max_kernel=5):
    """Function to sample a random architecture from the search space."""
    num_layers = random.randint(min_layers, max_layers)
    layer_dims = [(random.randint(min_filters, max_filters), random.randint(min_kernel, max_kernel)) for _ in range(num_layers)]
    return layer_dims

In [ ]:
def mutate_architecture(layer_dims, mutation_rate=0.1, min_layers=1, max_layers=5, min_filters=32, max_filters=512, min_kernel=3, max_kernel=5):
    """Function to mutate an architecture by randomly modifying some of its layer dimensions."""
    num_layers = len(layer_dims)
    for i in range(num_layers):
        if np.random.rand() < mutation_rate:
            layer_dims[i] = (random.randint(min_filters, max_filters), random.randint(min_kernel, max_kernel))
    return layer_dims

In [ ]:
def breed_architectures(parent1, parent2, mutation_rate, min_layers, max_layers, min_filters, max_filters, min_kernel, max_kernel):
    """Function to breed two parent architectures to produce a child architecture."""
    child = []
    for i in range(len(parent1)):
        if np.random.rand() < 0.5:
            child.append(parent1[i])
        else:
            if i < len(parent2):
                child.append(parent2[i])
    return mutate_architecture(child, mutation_rate, min_layers, max_layers, min_filters, max_filters, min_kernel, max_kernel)


In [ ]:
#Train the model on the dataset and evaluate its performance.
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    """Function to train and evaluate a model."""
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    scores = model.evaluate(X_test, y_test, verbose=0)
    return scores[1]  # Return accuracy

In [ ]:
# 4) Define the genetic algorithm to evolve the architecture.
def genetic_algorithm(population_size=20, num_generations=10, mutation_rate=0.1,\
                      min_layers=1, max_layers=5, min_filters=32, max_filters=512,\
                      min_kernel=3, max_kernel=5):
    """Function to implement the genetic algorithm to evolve the architecture."""
    # Initialize population
    population = [sample_architecture(min_layers, max_layers, min_filters,\
                                      max_filters,min_kernel, max_kernel)\
                  for _ in range(population_size)]
    scores = []
    for i in range(num_generations):
        print("Generation {}/{}".format(i + 1, num_generations))
        new_population = []
        for j in range(population_size):
            # Select two random parents
            the_choice=np.random.choice(population_size, 2, replace=False)
            parent1=population[the_choice[0]]
            parent2=population[the_choice[1]]
            # Breed the parents to produce a child
            child = breed_architectures(parent1, parent2, mutation_rate,\
                                        min_layers,max_layers, min_filters,\
                                        max_filters, min_kernel, max_kernel)
            new_population.append(child)
        # Evaluate each model in the population
        scores = []
        for j, layer_dims in enumerate(new_population):
            model = build_model(layer_dims,input_shape=(28,28,1,))
            (X_train, y_train), (X_test, y_test)=tf.keras.datasets.mnist.load_data()
            score = train_and_evaluate_model(model, X_train, y_train, X_test, y_test)
            scores.append((layer_dims, score))
        # Sort the models by accuracy and select the best ones for the next generation
        scores.sort(key=lambda x: x[1], reverse=True)
        population = [x[0] for x in scores[:population_size]]
    # Return the best architecture
    return population[0]

In [ ]:
input_shape = (200, 100,3) 
# Run the genetic algorithm to evolve the architecture
best_architecture = genetic_algorithm()

Generation 1/10
1


ValueError: ignored